In [1]:
import pandas as pd
import numpy as np

In [3]:
dataset=pd.read_csv("CKD.csv")
dataset.head(5)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.0,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.0,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.0,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.0,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.0,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes


In [5]:
dataset.shape

(399, 25)

In [7]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset.head(5)

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.0,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,False,False,False,False,False,False,True,True,False,True
1,3.0,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,True,False,False,False,False,False,True,False,False,True
2,4.0,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,True,False,False,False,False,False,True,False,False,True
3,5.0,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,True,False,False,False,False,False,True,False,True,True
4,5.0,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,True,False,False,False,False,False,True,False,False,True


In [9]:
dataset.columns


Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [11]:
#Use iloc for position-based indexing (where -1 works for the last column).

#Use loc for label-based indexing (where you must reference the column by its name).

In [13]:
except_last_column=dataset.columns[:-1]
last_column=dataset.columns[-1]

In [15]:
dataset.loc[:,last_column].value_counts()

classification_yes
True     249
False    150
Name: count, dtype: int64

In [17]:
independent=dataset.loc[:,except_last_column]

In [19]:
dependent=dataset.loc[:,last_column]

In [21]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.20,random_state=0)

In [22]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
Logistic_param_Grid={'penalty':['l2'],'solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga']}
Logistic_Classifier=GridSearchCV(LogisticRegression(),Logistic_param_Grid,verbose=3,refit=True,n_jobs=-1)
Logistic_Classifier.fit(x_train,y_train)


Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             verbose=3)

In [65]:
re=Logistic_Classifier.cv_results_
table=pd.DataFrame(re)
#print(table)

print(f"Best Estimator:{Logistic_Classifier.best_estimator_}")
print(f"Best Params:{Logistic_Classifier.best_params_}")
Logistic_pred=Logistic_Classifier.predict(x_test)

from sklearn.metrics import confusion_matrix
Logistic_cm=confusion_matrix(y_test,Logistic_pred)
print("Confusion_Matricx")
print(Logistic_cm)

from sklearn.metrics import classification_report
Logistic_cr=classification_report(y_test,Logistic_pred)
print("Classification Report")
print(Logistic_cr)
from sklearn.metrics import f1_score
Logistic_f1=f1_score(y_test,Logistic_pred,average='weighted')
print(f"Logistic Cassification_Best_params:{Logistic_Classifier.best_params_}& f1_macro:{Logistic_f1}")

from sklearn.metrics import roc_auc_score
Logistic_roc_auc=roc_auc_score(y_test,Logistic_Classifier.predict_proba(x_test)[:,1])
print(f"Logistic Cassification_roc_auc:{Logistic_roc_auc}")

Best Estimator:LogisticRegression()
Best Params:{'penalty': 'l2', 'solver': 'lbfgs'}
Confusion_Matricx
[[32  0]
 [ 1 47]]
Classification Report
              precision    recall  f1-score   support

       False       0.97      1.00      0.98        32
        True       1.00      0.98      0.99        48

    accuracy                           0.99        80
   macro avg       0.98      0.99      0.99        80
weighted avg       0.99      0.99      0.99        80

Logistic Cassification_Best_params:{'penalty': 'l2', 'solver': 'lbfgs'}& f1_macro:0.9875303643724695
Logistic Cassification_roc_auc:1.0


In [67]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier_param_Grid={'criterion':['gini','entropy','log_loss'],'splitter':['best','random']}
DecisionTree_Classifier=GridSearchCV(DecisionTreeClassifier(),DecisionTreeClassifier_param_Grid,verbose=3,refit=True,n_jobs=-1)
DecisionTree_Classifier.fit(x_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [69]:
DecsionTree_re=DecisionTree_Classifier.cv_results_
table_DecsionTree=pd.DataFrame(DecsionTree_re)
#print(table_DecsionTree)

print(f"Best Estimator:{DecisionTree_Classifier.best_estimator_}")
print(f"Best Params:{DecisionTree_Classifier.best_params_}")
DecisionTreeClassifier_pred=DecisionTree_Classifier.predict(x_test)

from sklearn.metrics import confusion_matrix
DecisionTreeClassifier_cm=confusion_matrix(y_test,DecisionTreeClassifier_pred)
print("Confusion_Matricx")
print(DecisionTreeClassifier_cm)

from sklearn.metrics import classification_report
DecisionTreeClassifier_cr=classification_report(y_test,DecisionTreeClassifier_pred)
print("Classification Report")
print(DecisionTreeClassifier_cr)

from sklearn.metrics import f1_score
DecisionTreeClassifier_f1=f1_score(y_test,DecisionTreeClassifier_pred,average='weighted')
print(f"DecisionTreeClassifier_Best_params:{DecisionTree_Classifier.best_params_}& f1_macro:{DecisionTreeClassifier_f1}")

from sklearn.metrics import roc_auc_score
DecisionTreeClassifier_roc_auc=roc_auc_score(y_test,DecisionTree_Classifier.predict_proba(x_test)[:,1])
print(f"DecisionTreeClassifier roc_auc:{DecisionTreeClassifier_roc_auc}")

Best Estimator:DecisionTreeClassifier(criterion='log_loss', splitter='random')
Best Params:{'criterion': 'log_loss', 'splitter': 'random'}
Confusion_Matricx
[[32  0]
 [ 0 48]]
Classification Report
              precision    recall  f1-score   support

       False       1.00      1.00      1.00        32
        True       1.00      1.00      1.00        48

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

DecisionTreeClassifier_Best_params:{'criterion': 'log_loss', 'splitter': 'random'}& f1_macro:1.0
DecisionTreeClassifier roc_auc:1.0


In [71]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
SVM_Classifier_param_Grid={'kernel':['linear','poly','rbf','sigmoid']}
SVM_Classifier=GridSearchCV(SVC(probability=True),SVM_Classifier_param_Grid,verbose=3,refit=True,n_jobs=-1)
SVM_Classifier.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             verbose=3)

In [73]:
SVM_Classifier_re=SVM_Classifier.cv_results_
SVM_Classifier_table=pd.DataFrame(SVM_Classifier_re)
#print(SVM_Classifier_table)

print(f"Best Estimator:{SVM_Classifier.best_estimator_}")
print(f"Best Params:{SVM_Classifier.best_params_}")
SVM_Classifier_pred=SVM_Classifier.predict(x_test)

from sklearn.metrics import confusion_matrix
SVM_Classifier_cm=confusion_matrix(y_test,SVM_Classifier_pred)
print("Confusion_Matricx")
print(SVM_Classifier_cm)

from sklearn.metrics import classification_report
SVM_Classifier_cr=classification_report(y_test,SVM_Classifier_pred)
print("Classification Repot")
print(SVM_Classifier_cr)

from sklearn.metrics import f1_score
SVM_Classifier_f1=f1_score(y_test,SVM_Classifier_pred,average='weighted')
print(f"SVM_Classifier_Best_params:{SVM_Classifier.best_params_}& f1_macro:{SVM_Classifier_f1}")

from sklearn.metrics import roc_auc_score
SVM_Classifier_roc_auc=roc_auc_score(y_test,SVM_Classifier.predict_proba(x_test)[:,1])
print(f"SVM_Classifier roc_auc:{SVM_Classifier_roc_auc}")

Best Estimator:SVC(probability=True)
Best Params:{'kernel': 'rbf'}
Confusion_Matricx
[[32  0]
 [ 1 47]]
Classification Repot
              precision    recall  f1-score   support

       False       0.97      1.00      0.98        32
        True       1.00      0.98      0.99        48

    accuracy                           0.99        80
   macro avg       0.98      0.99      0.99        80
weighted avg       0.99      0.99      0.99        80

SVM_Classifier_Best_params:{'kernel': 'rbf'}& f1_macro:0.9875303643724695
SVM_Classifier roc_auc:1.0


In [75]:
from sklearn.ensemble import RandomForestClassifier

RandomForest_Classifier_param_Grid={'n_estimators':[10,100,200,1000],'criterion':['gini','entropy']}
RandomForest_Classifier=GridSearchCV(RandomForestClassifier(),RandomForest_Classifier_param_Grid,verbose=3,refit=True,n_jobs=-1)
RandomForest_Classifier.fit(x_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'n_estimators': [10, 100, 200, 1000]},
             verbose=3)

In [77]:
RandomForest_Classifier_re=RandomForest_Classifier.cv_results_
RandomForest_Classifier_table=pd.DataFrame(RandomForest_Classifier_re)
#print(RandomForest_Classifier_table)

print(f"Best Estimator:{RandomForest_Classifier.best_estimator_}")
print(f"Best Params:{RandomForest_Classifier.best_params_}")
RandomForest_Classifier_pred=RandomForest_Classifier.predict(x_test)

from sklearn.metrics import confusion_matrix
RandomForest_Classifier_cm=confusion_matrix(y_test,RandomForest_Classifier_pred)
print("Confusion_Matricx")
print(RandomForest_Classifier_cm)

from sklearn.metrics import classification_report
RandomForest_Classifier_cr=classification_report(y_test,RandomForest_Classifier_pred)
print("Classification Repot")
print(RandomForest_Classifier_cr)

from sklearn.metrics import f1_score
RandomForest_Classifier_f1=f1_score(y_test,RandomForest_Classifier_pred,average='weighted')
print(f"RandomForest_Classifier_Best_params:{RandomForest_Classifier.best_params_}& f1_macro:{RandomForest_Classifier_f1}")

from sklearn.metrics import roc_auc_score
RandomForest_Classifier_roc_auc=roc_auc_score(y_test,RandomForest_Classifier.predict_proba(x_test)[:,1])
print(f"RandomForest_Classifier roc_auc:{RandomForest_Classifier_roc_auc}")

Best Estimator:RandomForestClassifier(n_estimators=200)
Best Params:{'criterion': 'gini', 'n_estimators': 200}
Confusion_Matricx
[[31  1]
 [ 0 48]]
Classification Repot
              precision    recall  f1-score   support

       False       1.00      0.97      0.98        32
        True       0.98      1.00      0.99        48

    accuracy                           0.99        80
   macro avg       0.99      0.98      0.99        80
weighted avg       0.99      0.99      0.99        80

RandomForest_Classifier_Best_params:{'criterion': 'gini', 'n_estimators': 200}& f1_macro:0.9874652266404844
RandomForest_Classifier roc_auc:1.0


In [79]:
#'age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv','wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal','pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes', 'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes
       
       
       

In [81]:
age=(input("Age:"))
bp=float(input("bp:"))
al=float(input("al:"))
su=float(input("su:"))
bgr=float(input("bgr:"))
bu=float(input("bu:"))
sc=float(input("sc:"))
sod=float(input("sod:"))
pot=float(input("pot:"))
hrmo=float(input("hrmo:"))
pcv=float(input("pcv:"))
wc=float(input("wc:"))
rc=float(input("rc:"))
sg_b=float(input("sg_b:"))
sg_c=float(input("sg_c:"))
sg_d=float(input("sg_d:"))
sg_e=float(input("sg_e:"))
rbc_normal=float(input("rbc_normal:"))
pc_normal=float(input("pc_normal:"))
pcc_present=float(input("pcc_present:"))
ba_present=float(input("ba_present:"))
htn_yes_or_No=float(input("htn_yes/NO:"))
dm_yes_or_No=float(input("dm_yes/No:"))
cad_yes_or_No=float(input("cad_yes/No:"))
appet_yes_or_No=float(input("appet_yes/No:"))
pe_yes_or_No=float(input("cad_yes/No:"))
ane_yes_or_No=float(input("ane_yes/No:"))

Age: 25
bp: 234
al: 789
su: 1567
bgr: 4356
bu: 2778
sc: 33333
sod: 56777
pot: 3333
hrmo: 23456
pcv: 10999
wc: 56665
rc: 3217
sg_b: 1234
sg_c: 8798
sg_d: 4567
sg_e: 3333
rbc_normal: 2323
pc_normal: 1899
pcc_present: 0999
ba_present: 55555
htn_yes/NO: 1
dm_yes/No: 1
cad_yes/No: 1
appet_yes/No: 1
cad_yes/No: 1
ane_yes/No: 0


In [83]:
Future_Prediction=DecisionTree_Classifier.predict([[age, bp, al, su, bgr, bu, sc, sod, pot, hrmo, pcv,
       wc, rc, sg_b, sg_c, sg_d, sg_e, rbc_normal, pc_normal,
       pcc_present, ba_present, htn_yes_or_No, dm_yes_or_No, cad_yes_or_No,
       appet_yes_or_No, pe_yes_or_No, ane_yes_or_No]])

In [85]:
Future_Prediction

array([ True])

In [87]:
import pickle
filename="CassificationAssignment.sav"
pickle.dump(DecisionTree_Classifier,open(filename,'wb'))